# Introduction
train and test data are measured with same roll resistance (lever position 3) and wheel pressure.


# Analyze linearity of train vs test data
Question: Is the velocity a good feature to predict the virtual power? To find out we create a linear/polynomial model or KNN regression model from the app data and calculate a score for each tacx cadence value to see whether the model predicts the velocity good enough.
First let us see (plot) and calculate (relation of cadence/velocity is constant per gear)  the training data.

_Train data (app data):
* X of app data set -> cadence
* Y -> velocity

## Pre processing


In [219]:
import pandas as pd
import numpy as np
from src.tcx import read_tcx

tcx: dict = read_tcx('test/tcx/cadence_1612535177298-gear7.tcx')


trackpoints: dict = tcx['TrainingCenterDatabase']['Activities']['Activity']['Lap']['Track']
trackpoint_dict_list = list(trackpoints.values())[0]

tcx_df: pd.DataFrame = pd.DataFrame.from_records(trackpoint_dict_list)

tcx_df['DistanceMeters'] = tcx_df['DistanceMeters'].apply(lambda x: float(x))
tcx_df['DistanceMeters']

# print(tcx_df['Extensions'])
tcx_df['Ext.Watts'] = [extension_dict['TPX']['Watts'] for extension_dict in tcx_df['Extensions']]
tcx_df['Ext.Speed'] = [extension_dict['TPX']['Speed'] for extension_dict in tcx_df['Extensions']]


tcx_df['DistanceMeters-delta'] = pd.Series(np.subtract(tcx_df['DistanceMeters'].iloc[1:].to_numpy(),tcx_df['DistanceMeters'].iloc[0:-1].to_numpy()), index=range(1, len(tcx_df)))

tcx_df[['DistanceMeters', 'Ext.Speed', 'Ext.Watts', 'DistanceMeters-delta']].head(10)

# df = pd.json_normalize([{'activity': {'id': 'tacx 12.11.'}}, {'activity': {'id': 'olli'}}]);
# df

,DistanceMeters,Ext.Speed,Ext.Watts,DistanceMeters-delta
0,5.31240,5.312399999999999,0,NaN
1,11.53350,6.2211,0,6.22110
2,17.75460,6.2211,0,6.22110
3,23.94075,6.18615,0,6.18615
4,30.23175,6.291,0,6.29100
5,36.62760,6.395849999999999,0,6.39585
6,43.02345,6.395849999999999,0,6.39585
7,49.31445,6.291,0,6.29100
8,55.46565,6.1512,0,6.15120
9,61.40715,5.9415000000000004,0,5.94150


## Linear regression